## Expanding function to be used to expand theorems to help comprehension

In [1]:
from adapt_utils import replace_symbols

from tree_parser import *
from tree_parser import proof_step

import networkx as nx

import matplotlib
import matplotlib.pyplot as plt

import copy

import time

from adapt_utils import replace_symbols
from my_utils import print_proof_props_graph, get_proof_steps, print_ident_proof, print_proof_linear_steps
from my_utils import get_proof_steps_graph, print_proof_steps_graph

import pickle



In [7]:
%%time

text = file_contents()
database = meta_math_database(text,n=3500)

included 5555695 tokens from set.mm
proposition: 3500CPU times: user 10.4 s, sys: 349 ms, total: 10.8 s
Wall time: 10.9 s


In [330]:
context_values = dict()

for prop in database.propositions_list:
    for k, v in prop.f.items():
        if k in context_values:
            if v.statement != context_values[k]:
                print(k, v.statement, context_values[k])
                break
        else:
            context_values[k] = v.statement

In [342]:
def eval_tree(tree):
    if tree.value in context_values: 
        return context_values[tree.value]
    
    prop = database.propositions[tree.value]
    assert len(tree.leaves)==len(prop.hyps)
    
    replacement_dict = {prop.hyps[i].variable: eval_tree(tree.leaves[i]) for i in range(len(prop.hyps)) if prop.hyps[i].type == "f"} #find the replacement rules for the variables
    #print 'replacement_dict', replacement_dict, 'applied to', prop.statement
    return string_replace(prop.statement[1:],replacement_dict)

def tree2str(tree):
    evaluated_tree = eval_tree(tree)
    return replace_symbols(" ".join(evaluated_tree))

In [825]:
class PTree:
    def __init__(self, value, leaves):
        self._default_value = value
        self._default_leaves = leaves
        self._connected_ptree = None
        
    def __repr__(self):
        return self.__str__()

    def __str__(self):
        return '<PTree: ' + self.stringify() + '>'
    
    @property
    def value(self):
        if self._connected_ptree:
            return self._connected_ptree.value
        return self._default_value
    
    @property
    def leaves(self):
        if self._connected_ptree:
            return self._connected_ptree.leaves
        
        return self._default_leaves

    def stringify(self):
        if self.value is None: 
            return "???"  # for the beam search
        
        out = self.value + "("
        dontaddcomma = True
        for l in self.leaves:
            if not dontaddcomma: 
                out += ","
            else: 
                dontaddcomma = False
            out += l.stringify()
        out += ")"
        return out
    
    @property
    def statement(self):
        return tree2str(self)
            
    def connect(self, target_tree):
        assert target_tree.value == self.value, f"{target_tree.value} {self.value}"
        assert len(target_tree.leaves) == len(self.leaves)
        
        #We can connect direct as the command below because other trees here are references by the symbols, 
        #not by the tree itself, since the arrangements of symbols are diferent.
        #self._connected_ptree = target_tree <- This doesnt work due that
        for this_leaf, target_leaf in zip(self.leaves, target_tree.leaves):
            this_leaf._connected_ptree = target_leaf
            
    def disconnect(self):
        for this_leaf in self.leaves:
            this_leaf._connected_ptree = None

# class PTreePlaceholder:
#     def __init__(self):
#         self.tree = None
    
#     @property
#     def value(self):
#         return self.tree.value
    
#     @property
#     def leaves(self):
#         return self.tree.leaves
         
#     def stringify(self):
#         return self.tree.stringify()
    
#     def __repr__(self):
#         return self.__str__()

#     def __str__(self):
#         return '<PTreePlaceholder: ' + self.stringify() + '>'

In [826]:
def tree2ptree(tree, ph_dict={}):

    new_leaves = []
    for l in tree.leaves:
        if str(l) not in ph_dict:
            ph_dict[str(l)] = tree2ptree(l, ph_dict)
        new_leaves.append(ph_dict[str(l)])
    
    return PTree(tree.value, new_leaves)

In [827]:
prop = database.propositions["bitr4i"]
tree2ptree(prop.tree).get_statement()

AttributeError: 'PTree' object has no attribute 'get_statement'

In [828]:
# def connect_trees(output_trees, input_trees, trees_to_update=[]):
    
#     assert len(output_trees) == len(input_trees)
    
#     repdict = {}
    
#     for output_tree, input_tree in zip(output_trees, input_trees):
       
#         raw_tree = input_tree
#         rep_tree = output_tree

#         for pos in raw_tree.breadth_first_position_list():
#             raw_subtree = raw_tree.tree_at_position(pos)
#             rep_subtree = rep_tree.tree_at_position(pos)
#             if raw_subtree.value != rep_subtree.value:
#                 repdict[raw_subtree.value] = rep_subtree

#         input_tree.replace(repdict)
        
#     #Update additional trees in the list
#     for t in trees_to_update:
#         t.replace(repdict)
        
 


class PNode:
    def __init__(self, prop):
        ph_dict = {}
        
        self._prop = prop
        self.output_tree = tree2ptree(prop.tree, ph_dict)
        self.input_trees = [tree2ptree(h.tree, ph_dict) for h in prop.hyps if h.type == 'e']
        self.output = None
        self.inputs = []
    
    @property
    def statement(self):
        return self.output_tree.statement
    
    def expand(self):
        #Return set of PNodes
        #Return the root PNode
        pass

In [829]:
prop1 = database.propositions["bitr4i"]
pn1 = PNode(prop1)
pn1.statement

'( 𝜑 ↔ 𝜒 )'

In [830]:
prop2 = database.propositions["3bitri"]
pn2 = PNode(prop2)
pn2.statement

'( 𝜑 ↔ 𝜃 )'

In [831]:
prop3 = database.propositions["albii"]
pn3 = PNode(prop3)
pn3.statement

'( ∀ x 𝜑 ↔ ∀ x 𝜓 )'

In [832]:
prop4 = database.propositions["pm4.71"]
pn4 = PNode(prop4)
pn4.statement

'( ( 𝜑 → 𝜓 ) ↔ ( 𝜑 ↔ ( 𝜑 ∧ 𝜓 ) ) )'

In [833]:
prop5 = database.propositions["dfss"]
pn5 = PNode(prop5)
pn5.statement

'( A ⊆ B ↔ A = ( A ∩ B ) )'

In [834]:
prop6 = database.propositions["eqeq2i"]
pn6 = PNode(prop6)
pn6.statement

'( C = A ↔ C = B )'

In [835]:
prop7 = database.propositions["abeq2"]
pn7 = PNode(prop7)
pn7.statement

'( A = { x | 𝜑 } ↔ ∀ x ( x ∈ A ↔ 𝜑 ) )'

In [836]:
pn1.input_trees[0].connect(pn2.output_tree)
pn1.statement

'( 𝜑 ↔ 𝜒 )'

In [837]:
pn2.input_trees[0].connect(pn5.output_tree)
pn2.statement

'( A ⊆ B ↔ 𝜃 )'

In [838]:
pn1.statement

'( A ⊆ B ↔ 𝜒 )'

In [839]:
prop0 = database.propositions["dfss2"]
pn0 = PNode(prop0)
pn0.statement

'( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )'

In [840]:
prop1 = database.propositions["bitr4i"]
pn1 = PNode(prop1)
pn1.statement

'( 𝜑 ↔ 𝜒 )'

In [841]:
pn1.output_tree.connect(pn0.output_tree)

In [842]:
pn1.statement

'( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )'

In [843]:
pn1.input_trees[0].statement

'( A ⊆ B ↔ 𝜓 )'

In [844]:
pn1.input_trees[1].statement

'( ∀ x ( x ∈ A → x ∈ B ) ↔ 𝜓 )'

In [845]:
prop2 = database.propositions["3bitri"]
pn2 = PNode(prop2)
pn2.statement

'( 𝜑 ↔ 𝜃 )'

In [846]:
pn2.output_tree.connect(pn1.input_trees[0])

In [847]:
pn2.statement

'( A ⊆ B ↔ 𝜓 )'

In [822]:
pn2.input_trees[2].statement

'( 𝜒 ↔ 𝜓 )'

In [848]:
prop22 = database.propositions["df-in"]
pn22 = PNode(prop22)
pn22.statement

'( A ∩ B ) = { x | ( x ∈ A ∧ x ∈ B ) }'

In [851]:
prop23 = database.propositions["eqeq2i"]
pn23 = PNode(prop23)
pn23.statement

'( C = A ↔ C = B )'

In [853]:
pn23.input_trees[0].connect(pn22.output_tree)

In [854]:
pn23.statement

'( C = ( A ∩ B ) ↔ C = { x | ( x ∈ A ∧ x ∈ B ) } )'

In [855]:
pn2.input_trees[2].connect(pn23.output_tree)
pn2.statement

'( A ⊆ B ↔ C = { x | ( x ∈ A ∧ x ∈ B ) } )'

In [ ]:
Pelo bug de cima isso aqui vai dar muita zica

In [823]:
prop5 = database.propositions["dfss"]
pn5 = PNode(prop5)
pn5.statement

'( A ⊆ B ↔ A = ( A ∩ B ) )'

In [824]:
pn5.output_tree.connect(pn2.input_trees[0])
pn5.statement

'( A ⊆ B ↔ 𝜓 )'

In [730]:
pn2.input_trees[0]._connected_ptree

<PTree: wb(wss(cA(),cB()),wceq(cA(),cin(cA(),cB())))>

In [731]:
pn2.input_trees[0]

<PTree: wb(wss(cA(),cB()),wceq(cA(),cin(cA(),cB())))>

In [733]:
pn2.output_tree

<PTree: wb(wph(),wth())>

In [707]:
pn2.input_trees[1].connect(pn6.output_tree)
pn2.describe()

'( A ⊆ B ↔ 𝜃 )'

In [711]:
pn2.input_trees[2].connect(pn7.output_tree)
pn2.describe()

'( A ⊆ B ↔ ∀ x ( x ∈ A ↔ 𝜑 ) )'

In [712]:
pn1.describe()

'( A ⊆ B ↔ 𝜒 )'

o problema aqui são as variaveis que nao tem origem nenhuma, teria que propaga-las para baixo. Talvez o esquema seja usar mesmo o que ja tava fazendo com os replace dict

In [704]:
pn2.output_tree

<PTree: wb(wss(cA(),cB()),wth())>

In [638]:
pn3.input_trees[0].connect(pn4.output_tree)

In [510]:
pn3.describe()

'( ∀ x ( 𝜑 → 𝜓 ) ↔ ∀ x ( 𝜑 ↔ ( 𝜑 ∧ 𝜓 ) ) )'

In [512]:
pn1.input_trees[1].connect(pn3.output_tree)

In [513]:
pn1.describe()

'( 𝜑 ↔ ∀ x ( 𝜑 → 𝜓 ) )'

In [515]:
pn1.input_trees[0].connect(pn2.output_tree)

In [526]:
pn1.output_tree

<PTree: wb(wph(),wal(wi(wph(),wps()),vx()))>

In [521]:
pn2.input_trees[0].connect(pn5.output_tree)

In [524]:
pn2.describe()

'( A ⊆ B ↔ 𝜃 )'

'( A ⊆ B ↔ A = ( A ∩ B ) )'

In [497]:
pn1.input_trees[0].connect(pn2.output_tree)

In [498]:
pn1.describe()

'( 𝜑 ↔ 𝜒 )'

In [499]:
pn1.input_trees[1].connect(pn3.output_tree)

In [500]:
pn1.describe()

'( 𝜑 ↔ ∀ x 𝜑 )'

In [480]:
pn2.output_tree

<PTree: wb(wph(),wth())>

In [427]:
connect_trees([pn2.output_tree, pn3.output_tree], pn1.input_trees, [pn1.output_tree])

In [428]:
pn1.input_trees

[<Tree: wb(wph(),wth())>, <Tree: wb(wal(wph(),vx()),wal(wps(),vx()))>]

In [429]:
pn1.output_tree

<Tree: wb(wph(),wal(wph(),vx()))>

In [430]:
pn1.describe()

'( 𝜑 ↔ ∀ x 𝜑 )'

In [379]:
prop = database.propositions["dfss2"]
pn = PNode(prop)
pn.describe()

'( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )'

In [373]:
pn.inputs[0].leaves[0]

<Tree: wph()>

In [360]:
prop.tree.copy().leaves[1] == prop.hyps[-1].tree.leaves[0]

True

In [356]:
prop.hyps[-1].tree.leaves[0]

<Tree: wch()>

In [349]:
[h. for h in prop.hyps if h.type == 'e']

[]

In [4]:
#%%time
#database = pickle.load(open("database10000.pkl", "rb"))

In [5]:
#pickle.dump(database, open("database10000.pkl", "wb"))

In [346]:
#print_proof_linear_steps("dfss2", database)

In [6]:
#1. Olhar pro passo
#2. Pegar as variaveis F do teorema correspondente ao passo
#3. Preencher dentro dos passos dentro da prova do teorema

In [347]:
#print_proof_linear_steps("dfss2", database)

In [289]:
def populate_post_entails(step):
    #Get all steps. This is necessary because hypothesis are not natively within entails proof_steps
    all_steps = []
    for pstep in step.entails_proof_steps:
        if pstep not in all_steps:
            all_steps.append(pstep)

        for prior_pstep in pstep._prior_entails:
            if prior_pstep not in all_steps:
                all_steps.append(prior_pstep)

    for child_step in all_steps:
        child_step._post_entail = None
        
    for child_step in all_steps:
        for gchild_step in child_step._prior_entails:
            gchild_step._post_entail = child_step

In [ ]:
class PNode:
    pass
        
        
def create_node(prop):
    node = PNode()
    

In [299]:
prop.tree

<Tree: wb(wph(),wch())>

In [295]:
prop.label

'bitr4i'

In [300]:
prop = database.propositions["dfss2"]

In [311]:
" ".join(prop.statement)

'|- ( A C_ B <-> A. x ( x e. A -> x e. B ) )'

In [309]:
prop.tree

<Tree: wb(wss(cA(),cB()),wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx()))>

In [ ]:
tree_values_dict = {
    'wb': '<->',
    'cA': 'A',
    'cB': 'B',
    'wss': 'C_',
    'wal': 'A.',
    'wi': '->',
    'wcel': 'e.'
    
}

In [329]:
context_values

{'wph': ['ph'],
 'wps': ['ps'],
 'wch': ['ch'],
 'wth': ['th'],
 'wta': ['ta'],
 'wet': ['et'],
 'wze': ['ze'],
 'wsi': ['si'],
 'wrh': ['rh'],
 'wmu': ['mu'],
 'wla': ['la'],
 'wka': ['ka'],
 'vx.wal': ['x'],
 'vx.cv': ['x'],
 'cA.wceq': ['A'],
 'cB.wceq': ['B'],
 'vx.tru': ['x'],
 'vy.tru': ['y'],
 'vx': ['x'],
 'vy': ['y'],
 'vz': ['z'],
 'vw': ['w'],
 'vt': ['t'],
 'vu': ['u'],
 'vv': ['v'],
 'v.vs': ['s'],
 'wcel.cA': ['A'],
 'wcel.cB': ['B'],
 'cbvex4v.vf': ['f'],
 'cbvex4v.vg': ['g'],
 'cA': ['A'],
 'cB': ['B'],
 'cC': ['C'],
 'cD': ['D'],
 'cF': ['F'],
 'cG': ['G'],
 'cE': ['E'],
 'cV': ['V'],
 'cR': ['R'],
 'cS': ['S'],
 'cW': ['W'],
 'cH': ['H'],
 'vs': ['s'],
 'cX': ['X'],
 'cT': ['T'],
 'cQ': ['Q'],
 'cY': ['Y'],
 'va': ['a']}

In [ ]:
def tree_to_string(tree, database, context):
    if tree.value in context.f: return context.f[tree.value].statement
    prop = database.propositions[tree.value]
    assert len(tree.leaves)==len(prop.hyps)
    replacement_dict = {prop.hyps[i].variable:tree_to_string(tree.leaves[i], database, context) for i in range(len(prop.hyps)) if prop.hyps[i].type == "f"} #find the replacement rules for the variables
    #print 'replacement_dict', replacement_dict, 'applied to', prop.statement
    return string_replace(prop.statement[1:],replacement_dict)

In [290]:
class ExpProof:
    
    def __init__(self, prop):
        self._prop = prop
        self.steps = {}
        self._next_id = 0
        
        #Populate child entails for every step
        populate_post_entails(prop)
        
        step2exp = dict() #Dictionary to store references from steps to expandable steps
        
        #Get all steps. This is necessary because hypothesis are not natively within entails proof_steps
        all_steps = []
        for pstep in prop.entails_proof_steps:
            if pstep not in all_steps:
                all_steps.append(pstep)
                
            for prior_pstep in pstep._prior_entails:
                if prior_pstep not in all_steps:
                    all_steps.append(prior_pstep)
        
        for step in all_steps:
            n_id = self.get_next_id()
            new_expproof_step = ExpProofStep(step, n_id, self, 0)
            self.steps[n_id] = new_expproof_step
            
            step2exp[step] = new_expproof_step
            
        #Set child step and parents steps
        none_count = 0 
        for s, e in step2exp.items():
            e.parents_steps = [step2exp[ss] for ss in s._prior_entails]
            if s._post_entail == None:
                none_count += 1
            else:
                e.child_step = step2exp[s._post_entail]
                
        assert none_count == 1, "There have to be only one none_count post_entail from the root step. none_count: " + none_count
            
            
    def get_next_id(self):
        n_id = self._next_id
        self._next_id += 1
        return n_id
            
    def print_proof(self):
        repr_str = ""
        
        print("[{}]".format(self._prop.label), "|-", replace_symbols(self._prop.tree.eval(database, self._prop)))
        
        for _step in self.steps.values():
            print(_step)
        

class ExpProofStep:
    
    def __init__(self, step, step_id, proof, depth=0):
        self._step = step
        self._step_id = step_id
        self._proof = proof
        self.depth = depth
        self.statement = replace_symbols(step.tree.eval(database, self._proof._prop))
        
        self.parents_steps = []
        self.child_step = None
        
        self._expanded = False
    
    def expand(self):
        if self._step.prop.type == "e":
            return
        
        #Set these flags to prevent bugs of calling expanded twice in the same step
        if self._expanded:
            raise Exception("Step already expanded.")  
        self._expanded = True
        
        step2exp = dict() #Dictionary to store references from steps to expandable steps
        
        expanded_steps = []
        for step in expand_proof_step(self._step):
            n_id = self._proof.get_next_id()
            new_expproof_step = ExpProofStep(step, n_id, self._proof, self.depth+1)
            expanded_steps.append(new_expproof_step)
            
            step2exp[step] = new_expproof_step
           
        
        
        
        
            
        #Set child step and parents steps
        none_count = 0 
        for s, e in step2exp.items():
            e.parents_steps = [step2exp.get(ss, ss) for ss in s._prior_entails]
            if s._post_entail == None:
                none_count += 1
            else:
                e.child_step = step2exp[s._post_entail]
                
        assert none_count == 1, "There have to be only one none_count post_entail from the root step. none_count: " + none_count    
            
            
            
        #Set hypothesis steps
#         hyps2steps = dict()
#         e_hyps = [h for h in self._step.prop.hyps if h.type == 'e']
#         assert len(e_hyps) == len(self._step._prior_entails), print(len(e_hyps), len(self._step._prior_entails))
        
#         for raw_hyp, rep_hyp in zip(e_hyps, step._prior_entails):
#             hyps2steps[raw_hyp] = rep_hyp
        
        #for step in expanded_steps:
            
            
        
        
        return expanded_steps
    
    def __repr__(self):
        return self.describe()
    
    def __str__(self):
        return self.describe()
        
    def describe(self):
        return " ".join([
            str(self._step_id),
            "---" * (self.depth+1),
            "[{}]".format(self._step.prop.label),
            str([s._step_id for s in self.parents_steps]),
            "|-",
            self.statement
        ])
        
def get_step_replace_dict(step):
    
    e_hyps = [h for h in step.prop.hyps if h.type == 'e']
    assert len(e_hyps) == len(step._prior_entails), print(len(e_hyps), len(step._prior_entails))
    
    repdict = {}
    
    for raw_hyp, rep_hyp in zip(e_hyps, step._prior_entails):
       
        raw_tree = raw_hyp.tree
        rep_tree = rep_hyp.tree

        for pos in raw_tree.breadth_first_position_list():
            raw_subtree = raw_tree.tree_at_position(pos)
            rep_subtree = rep_tree.tree_at_position(pos)
            if raw_subtree.value != rep_subtree.value:
                repdict[raw_subtree.value] = rep_subtree
        
    return repdict

def expand_proof_step(root_step):
    
    expanded_steps = []
    
    if not hasattr(root_step, "replace_dict_list"):
        root_step.replace_dict_list = [get_step_replace_dict(root_step)]
        
    step2exp = dict() #Store references from steps to expanded steps to populate 
    
    for child_step in root_step.prop.entails_proof_steps:
        #Take the original prop tree
        exp_tree = child_step.prop.tree.copy()
        
        #Take the step transformation that was used to the raw step to the replace step
        replace_dict_list = [get_step_replace_dict(child_step)]
            
        #Get the root step replace dicts so we can iterativelly expand the steps
        replace_dict_list.extend(root_step.replace_dict_list)
        
        for rep_dict in replace_dict_list:
            exp_tree = exp_tree.replace(rep_dict)
        
        prior_statements = child_step.prior_statements if hasattr(child_step, "prior_statements") else []
        
        exp_step = proof_step(exp_tree, root_step.context, child_step.prop, prior_statements)
        exp_step.replace_dict_list = replace_dict_list
        
        expanded_steps.append(exp_step)
        
        step2exp[child_step] = exp_step
        
    #Populate _prior_entails
    for child_step in root_step.prop.entails_proof_steps:
        step2exp[child_step]._prior_entails = [step2exp.get(cc, cc) for cc in child_step._prior_entails]
    
        
    return expanded_steps

In [291]:
prop = database.propositions["bitr4i"]
ep = ExpProof(prop)
ep.print_proof()

[bitr4i] |- ( 𝜑 ↔ 𝜒 )
0 --- [bicomi] [1] |- ( 𝜓 ↔ 𝜒 )
1 --- [bitr4i.2] [] |- ( 𝜒 ↔ 𝜓 )
2 --- [bitri] [3, 0] |- ( 𝜑 ↔ 𝜒 )
3 --- [bitr4i.1] [] |- ( 𝜑 ↔ 𝜓 )


In [292]:
eee = ep.steps[2].expand()
eee

AttributeError: 'proof_step' object has no attribute '_post_entail'

In [262]:
eee[1]._step._prior_entails[1]

In [268]:
prop = database.propositions["bitr4i"]
populate_post_entails(prop)

In [269]:
for exp_step in prop.entails_proof_steps:
    print("--- [{}]".format(exp_step.prop.label), len(exp_step._prior_entails), "|-", replace_symbols(exp_step.tree.eval(database, root_step.context)))

print()

root_step = prop.entails_proof_steps[-1]
exp_proof = expand_proof_step(root_step)

print("[{}]".format(root_step.prop.label), "|-", replace_symbols(root_step.tree.eval(database, root_step.context)))

for exp_step in exp_proof:
    print("--- [{}]".format(exp_step.prop.label), len(exp_step._prior_entails), "|-", replace_symbols(exp_step.tree.eval(database, root_step.context)))

KeyError: 'wps'

In [10]:
#Verificar como utilizar a lista previa de replace dicts

prop = database.propositions["impbii"]
root_step = prop.entails_proof_steps[-1]

replace_dict_list = [get_step_replace_dict(root_step)]

print("[{}]".format(root_step.prop.label), "|-", replace_symbols(root_step.tree.eval(database, root_step.context)))
print()

for child_step in root_step.prop.entails_proof_steps:
    new_tree = child_step.tree.copy()#.replace(replace_dict_list[0])
    
    for rep_dict in replace_dict_list:
        new_tree = new_tree.replace(rep_dict)
    
    #.replace(step_rep_dict)
    print("--- [{}]".format(exp_step.prop.label), "|-", replace_symbols(new_tree.eval(database, prop)))
    
    
    #print(get_step_replace_dict(child_step), child_step.prop.tree)
    #print(replace_symbols(child_step.tree.copy().replace(step_rep_dict).eval(database, prop)))
   # print()

[mp2] |- ( 𝜑 ↔ 𝜓 )



NameError: name 'exp_step' is not defined

In [11]:
#[p.label for p in database.propositions_list[-300:]]

In [12]:
root_step.prop.hyps[-1].type

'e'

In [73]:
root_step.prop.hyps[-1]

<e_hypothesis bitr4i.2: |- |-(ch<->ps)>

In [74]:
root_step.prop.entails_proof_steps[-1]._prior_entails[1]._prior_entails[0].prop == root_step.prop.hyps[-1]

True

In [76]:
dir(root_step)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_prior_entails',
 '_prior_statements',
 'applicable_propositions',
 'context',
 'descendants',
 'height',
 'prop',
 'replace_dict_list',
 'summarize',
 'tree',
 'unconstrained',
 'vclass']

In [13]:
root_step.prop.hyps[-1].tree

<Tree: wi(wph(),wi(wps(),wch()))>

In [23]:
get_step_replace_dict(root_step)

{'wph': <Tree: wss(cA(),cB())>,
 'wps': <Tree: wal(wb(wcel(cv(vx()),cA()),wa(wcel(cv(vx()),cA()),wcel(cv(vx()),cB()))),vx())>,
 'wch': <Tree: wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx())>}

In [27]:
prop = database.propositions["dfss2"]

for exp_step in prop.entails_proof_steps:
    print("--- [{}]".format(exp_step.prop.label), len(exp_step._prior_entails), "|-", replace_symbols(exp_step.tree.eval(database, root_step.context)))

print()

root_step = prop.entails_proof_steps[-1]
exp_proof = expand_proof_step(root_step)

print("[{}]".format(root_step.prop.label), "|-", replace_symbols(root_step.tree.eval(database, root_step.context)))
print()

for exp_step in exp_proof:
    print("--- [{}]".format(exp_step.prop.label), "|-", replace_symbols(exp_step.tree.eval(database, root_step.context)))

--- [dfss] 0 |- ( A ⊆ B ↔ A = ( A ∩ B ) )
--- [df-in] 0 |- ( A ∩ B ) = { x | ( x ∈ A ∧ x ∈ B ) }
--- [eqeq2i] 1 |- ( A = ( A ∩ B ) ↔ A = { x | ( x ∈ A ∧ x ∈ B ) } )
--- [abeq2] 0 |- ( A = { x | ( x ∈ A ∧ x ∈ B ) } ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )
--- [3bitri] 3 |- ( A ⊆ B ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )
--- [pm4.71] 0 |- ( ( x ∈ A → x ∈ B ) ↔ ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )
--- [albii] 1 |- ( ∀ x ( x ∈ A → x ∈ B ) ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )
--- [bitr4i] 2 |- ( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )

[bitr4i] |- ( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )

--- [bicomi] |- ( ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) ↔ ∀ x ( x ∈ A → x ∈ B ) )
--- [bitri] |- ( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )


In [21]:
prop.entails_proof_steps[-1]._prior_entails

In [22]:
exp_proof[1]._prior_entails

[]

In [21]:
root_step = exp_proof[-1]

exp_proof = expand_proof_step(root_step)

print("[{}]".format(root_step.prop.label), "|-", replace_symbols(root_step.tree.eval(database, root_step.context)))
print()

for exp_step in exp_proof:
    print("--- [{}]".format(exp_step.prop.label), "|-", replace_symbols(exp_step.tree.eval(database, root_step.context)))

[bitri] |- ( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )

--- [sylbb] |- ( A ⊆ B → ∀ x ( x ∈ A → x ∈ B ) )
--- [sylbbr] |- ( ∀ x ( x ∈ A → x ∈ B ) → A ⊆ B )
--- [impbii] |- ( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )


In [22]:
root_step = exp_proof[-1]

exp_proof = expand_proof_step(root_step)

print("[{}]".format(root_step.prop.label), "|-", replace_symbols(root_step.tree.eval(database, root_step.context)))
print()

for exp_step in exp_proof:
    print("--- [{}]".format(exp_step.prop.label), "|-", replace_symbols(exp_step.tree.eval(database, root_step.context)))

[impbii] |- ( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )

--- [impbi] |- ( ( A ⊆ B → ∀ x ( x ∈ A → x ∈ B ) ) → ( ( ∀ x ( x ∈ A → x ∈ B ) → A ⊆ B ) → ( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) ) ) )
--- [mp2] |- ( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )


In [23]:
root_step = exp_proof[-1]

exp_proof = expand_proof_step(root_step)

print("[{}]".format(root_step.prop.label), "|-", replace_symbols(root_step.tree.eval(database, root_step.context)))
print()

for exp_step in exp_proof:
    print("--- [{}]".format(exp_step.prop.label), "|-", replace_symbols(exp_step.tree.eval(database, root_step.context)))

[mp2] |- ( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )

--- [ax-mp] |- ( ( ∀ x ( x ∈ A → x ∈ B ) → A ⊆ B ) → ( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) ) )
--- [ax-mp] |- ( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )


In [91]:
root_step = database.propositions["impbii"].entails_proof_steps[-1]

In [92]:
step_rep_dict = get_step_replace_dict(root_step)
step_rep_dict

{'wph': <Tree: wi(wph(),wps())>,
 'wps': <Tree: wi(wps(),wph())>,
 'wch': <Tree: wb(wph(),wps())>}

In [93]:
root_step.prop.entails_proof_steps[-2].tree.copy().replace(step_rep_dict).eval(database, database.propositions["impbii"])

'( ( ps -> ph ) -> ( ph <-> ps ) )'

In [94]:
database.propositions["impbii"].entails_proof_steps[-2].tree.eval(database, database.propositions["impbii"])

'( ( ph -> ps ) -> ( ( ps -> ph ) -> ( ph <-> ps ) ) )'

In [95]:
database.propositions["impbii"]

In [96]:
done_steps = []

[mp2] |- ( 𝜑 ↔ 𝜓 )

--- [ax-mp] |- 𝜓
--- [ax-mp] |- 𝜓


In [98]:
root_step = exp_proof[-1]
done_steps.append(root_step)
exp_proof = expand_proof_step(root_step)

print("[{}]".format(root_step.prop.label), "|-", replace_symbols(root_step.tree.eval(database, root_step.context)))
print()

for exp_step in exp_proof:
    print("--- [{}]".format(exp_step.prop.label), "|-", replace_symbols(exp_step.tree.eval(database, root_step.context)))

2 0


AssertionError: None

In [ ]:
for child_step in root_step.prop.entails_proof_steps:
        #exp_tree = child_step.prop.tree.copy()
        exp_tree = child_step.tree.copy()
        
        #Check if the child step tree has a replace dict list (axioms dont have it I think)
        #replace_dict_list = child_step.replace_dict_list.copy()
        
        replace_dict_list = [get_step_replace_dict(child_step)]
        
        
        #if hasattr(child_step.tree, "replace_dict_list"):
        #    exp_tree.replace_dict_list = child_step.tree.replace_dict_list.copy()
        #else:
            #exp_tree.replace_dict_list = []
            
        replace_dict_list.extend(root_step.replace_dict_list)
        
        for rep_dict in replace_dict_list:
            exp_tree.replace(rep_dict)
        
        prior_statements = child_step.prior_statements if hasattr(child_step, "prior_statements") else []
        
        exp_step = proof_step(exp_tree, root_step.context, child_step.prop, prior_statements)
        exp_step.replace_dict_list = replace_dict_list
        
        expanded_steps.append(exp_step)

In [ ]:
def expand_proof_step(root_step):

    #rep_dict = root_step.tree._rep_dict
    
    expanded_steps = []
    
    for child_step in root_step.prop.entails_proof_steps:
        #exp_tree = child_step.prop.tree.copy()
        exp_tree = child_step.tree.copy()
        
        #Check if the child step tree has a replace dict list (axioms dont have it I think)
        #replace_dict_list = child_step.replace_dict_list.copy()
        
        replace_dict_list = [get_step_replace_dict(child_step)]
        
        
        #if hasattr(child_step.tree, "replace_dict_list"):
        #    exp_tree.replace_dict_list = child_step.tree.replace_dict_list.copy()
        #else:
            #exp_tree.replace_dict_list = []
            
        replace_dict_list.extend(root_step.replace_dict_list)
        
        for rep_dict in replace_dict_list:
            exp_tree.replace(rep_dict)
        
        prior_statements = child_step.prior_statements if hasattr(child_step, "prior_statements") else []
        
        exp_step = proof_step(exp_tree, root_step.context, child_step.prop, prior_statements)
        exp_step.replace_dict_list = replace_dict_list
        
        expanded_steps.append(exp_step)
        
    return expanded_steps

In [6]:
#database.propositions["mp2"].tree.replace_dict_list

AttributeError: 'Tree' object has no attribute 'replace_dict_list'

In [7]:
database.propositions["impbii"].entails_proof_steps[-1].replace_dict_list

[]

In [9]:
database.propositions["impbii"].entails_proof_steps[-1].tree

<Tree: wb(wph(),wps())>

In [24]:
root_step = exp_proof[-1]
done_steps.append(root_step)
exp_proof = expand_proof_step(root_step)


print("[{}]".format(root_step.prop.label), "|-", replace_symbols(root_step.tree.eval(database, root_step.context)))
print()

for exp_step in exp_proof:
    print("--- [{}]".format(exp_step.prop.label), "|-", replace_symbols(exp_step.tree.eval(database, root_step.context)))

[mp2] |- ( 𝜑 ↔ 𝜒 )

--- [ax-mp] |- ( 𝜒 → 𝜒 )
--- [ax-mp] |- 𝜒


In [25]:
root_step = exp_proof[-1]
done_steps.append(root_step)
exp_proof = expand_proof_step(root_step)


print("[{}]".format(root_step.prop.label), "|-", replace_symbols(root_step.tree.eval(database, root_step.context)))
print()

for exp_step in exp_proof:
    print("--- [{}]".format(exp_step.prop.label), "|-", replace_symbols(exp_step.tree.eval(database, root_step.context)))
#print("--- [{}]".format(exp_proof[-1].prop.label), "|-", replace_symbols(exp_proof[-1].tree.eval(database, root_step.context)))

[ax-mp] |- 𝜒



In [26]:
root_step = exp_proof[-1]
done_steps.append(root_step)
exp_proof = expand_proof_step(root_step)


print("[{}]".format(root_step.prop.label), "|-", replace_symbols(root_step.tree.eval(database, root_step.context)))
print()

for exp_step in exp_proof:
    print("--- [{}]".format(exp_step.prop.label), "|-", replace_symbols(exp_step.tree.eval(database, root_step.context)))

IndexError: list index out of range

In [11]:
done_steps[0].replace_dict_list

[{'wph': <Tree: wss(cA(),cB())>,
  'wps': <Tree: wal(wb(wcel(cv(vx()),cA()),wa(wcel(cv(vx()),cA()),wcel(cv(vx()),cB()))),vx())>,
  'wch': <Tree: wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx())>}]

In [12]:
done_steps[1].replace_dict_list

[{'wph': <Tree: wph()>, 'wps': <Tree: wps()>, 'wch': <Tree: wch()>},
 {'wph': <Tree: wss(cA(),cB())>,
  'wps': <Tree: wal(wb(wcel(cv(vx()),cA()),wa(wcel(cv(vx()),cA()),wcel(cv(vx()),cB()))),vx())>,
  'wch': <Tree: wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx())>}]

In [13]:
done_steps[2].replace_dict_list

[{'wph': <Tree: wph()>, 'wps': <Tree: wch()>},
 {'wph': <Tree: wph()>, 'wps': <Tree: wps()>, 'wch': <Tree: wch()>},
 {'wph': <Tree: wss(cA(),cB())>,
  'wps': <Tree: wal(wb(wcel(cv(vx()),cA()),wa(wcel(cv(vx()),cA()),wcel(cv(vx()),cB()))),vx())>,
  'wch': <Tree: wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx())>}]

In [14]:
done_steps[3].replace_dict_list

[{'wph': <Tree: wph()>, 'wps': <Tree: wch()>},
 {'wph': <Tree: wph()>, 'wps': <Tree: wps()>, 'wch': <Tree: wch()>},
 {'wph': <Tree: wss(cA(),cB())>,
  'wps': <Tree: wal(wb(wcel(cv(vx()),cA()),wa(wcel(cv(vx()),cA()),wcel(cv(vx()),cB()))),vx())>,
  'wch': <Tree: wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx())>}]

In [17]:
database.propositions["mp2"].entails_proof_steps[-1].replace_dict_list

[]

In [18]:
database.propositions["mp2"].hyps

[<f_hypothesis wph: wff ph>,
 <f_hypothesis wps: wff ps>,
 <f_hypothesis wch: wff ch>,
 <e_hypothesis mp2.1: |- |-ph>,
 <e_hypothesis mp2.2: |- |-ps>,
 <e_hypothesis mp2.3: |- |-(ph->(ps->ch))>]